# SEO Notebook

In [1]:
"""A framework for conducting SEO investigations, using Google Spreadsheets for I/O."""

'A framework for conducting SEO investigations, using Google Spreadsheets for I/O.'

sanity checks

make sure gspread is installed

do

$sudo pip3 install gspread

and

oauth2client

do

$sudo pip install --upgrade oauth2client

and

google-api-python-client

do

$sudo pip3 install --upgrade google-api-python-client



In [2]:
import notebook_finder    # Allows .ipynb files to be loaded like filename.py modules
import goodsheet          # Handles OAuth login for all relevant Google services
from imp import reload    # Keeps in-memory functions reflecting your pipulate.ipynb edits
import shelve, requests

ImportError: No module named 'myauth'

In [ ]:
rows_to_batch = 10
cache_html = True
google_sheet_name = 'Top 5 Search Pages from each Property'
google_sheet_tab  = 'Sheet1'

In [ ]:
if __name__ == '__main__':
    import pipulate
    reload(pipulate)
    import pipulate # 2nd import intentional
    bad_api_message = "Can't reach Google APIs. Please try again later."
    
    # Test our connection to the spreadsheet and get its size.
    try:
        worksheet = goodsheet.connect.open(google_sheet_name).worksheet(google_sheet_tab)
        rows = worksheet.row_count
        cols = worksheet.col_count
        end_range = worksheet.get_addr_int(rows, cols)
        col_count = worksheet.col_count
    except:
        print(bad_api_message)
        raise SystemExit()

    # Create a dictionary of name-addressable functions from the Pipulate module.
    pipulate_funcs = [x for x in dir(pipulate) if x[0] is not '_']
    func_dict = {x.lower():eval('pipulate.%s' % x) for x in pipulate_funcs}

    # Row 1 contains column name values and may name pipulate functions.
    row1_range = 'A1:%s' % worksheet.get_addr_int(1, col_count)
    cell_range = worksheet.range(row1_range)
    col_names = [x.value.lower() for x in cell_range]

    # Split spreadsheet into a series of ranges (not yet in Excel-like A2:B10 notation).
    chunk_ranges = [(x+1, x+rows_to_batch) for x in list(range(rows)) if x%rows_to_batch == 0]
    
    # Every range chunk becomes its own chunk_range for batch updates.
    for chunk_dex, (row_start, row_end) in enumerate(chunk_ranges):

        # Create Excel-ranges for each chunk
        top_left = worksheet.get_addr_int(row_start+1, 1)
        lower_right = worksheet.get_addr_int(row_end+1, cols)
        range_string = "%s:%s" % (top_left, lower_right)

        # If it's the last chunk, beware odd number of rows!
        if chunk_dex+1 == len(chunk_ranges):
            range_string = "%s:%s" % (top_left, end_range)
        print("Pipulating chunk %s of %s (%s)..." % (len(chunk_ranges), range_string, chunk_dex+1))

        # We create a chunk_range for the chunk for both reading and writing-back values.
        try:
            chunk_range = worksheet.range(range_string)
        except:
            print(bad_api_message)

        # Now, we step through each cell in the current chunk.
        row_dict = {} #pipulation object
        for cell_dex, acell in enumerate(chunk_range):
            row, col, val = acell.row, acell.col, acell.value
            # Populate row_dict with key/value pairs for every column of row.
            row_dict[col_names[col-1]] = val
            if col%col_count == 0: #last row of column / ready to pipulate!
                print("Row: %s of %s" % (row, rows))
                response = None
                # Why fetch the HTML for a URL more than once, if you don't have to?
                if 'url' in row_dict:
                    with shelve.open('urls') as urls:
                        if cache_html == True and row_dict['url'] in urls.keys():
                            response = urls[row_dict['url']]
                        else:
                            try:
                                response = requests.get(row_dict['url'])
                                urls[row_dict['url']] = response
                            except requests.exceptions.RequestException as e:
                                print("HTTP request failed. Check URL.")
                                raise SystemExit()
                        # We make ENTIRE response object is made available to Pipulate functions.
                        row_dict['response'] = response
                # To pipulate a row, we step through the row_dict that now contains all data from row.
                for key, val in row_dict.items():
                    if not val: #only pipulate empty cells
                        # Only process columns that use Pipulate function names.
                        if key in [x.lower() for x in dir(pipulate) if x[0] is not '_']:
                            try:
                                success_code, new_text = func_dict[key](**row_dict) # SHAZAM! Think about it.
                            except:
                                print('Problem in Pipulate function: %s' % key)
                                raise SystemExit()
                            # Remember, we are no longer actually ON the cell that we need to update, so we
                            # figure out which cell should updated with the value we just produced.
                            row_start = cell_dex-cols+1
                            func_dex = col_names.index(key)
                            cell_to_update = row_start + func_dex
                            chunk_range[cell_to_update].value = new_text #updates the in-memory object
                            row_dict = {} # Blank the now-used row_dict just for good measure
        try:
            # Batch update Google Sheets with the modified chunk_range.
            worksheet.update_cells(chunk_range)
            print("Range updated.")
        except:
            print(bad_api_message)
    print("Pipulation complete!") #do a little dance


In [ ]:
%tb